In [ ]:
#  Célula 1: Instancie o objeto de dados MNIST e import as lib

Importe as bibliotecas necessárias
import tensorflow as tf
from d2l import tensorflow as d2l
import pdb
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

# Defina a classe MNIST para carregar os dados
class MNIST(d2l.DataModule):
    def __init__(self, batch_size=64):
        super().__init__()
        self.save_hyperparameters()
        self.train, self.val = tf.keras.datasets.mnist.load_data()
        
data = MNIST()


In [ ]:
# Célula 2: Verifique o tamanho dos conjuntos de treinamento e validação, bem como as dimensões das imagens
print(len(data.train[0]), len(data.val[0]))
print(data.train[0].shape)

# Pré-processamento dos dados
@d2l.add_to_class(MNIST)
def get_dataloader(self, train):
    data = self.train if train else self.val
    process = lambda X, y: (tf.expand_dims(X, axis=3) / 255, tf.cast(y, dtype='int32'))
    resize_fn = lambda X, y: (tf.image.resize_with_pad(X, *self.resize), y)
    shuffle_buf = len(data[0]) if train else 1
    return tf.data.Dataset.from_tensor_slices(process(*data)).batch(self.batch_size).shuffle(shuffle_buf)


In [ ]:
# Célula 3: Verifique as dimensões de um mini lote de dados
X, y = next(iter(data.train_dataloader()))
print(X.shape, X.dtype, y.shape, y.dtype)

# Implemente o modelo MLP
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='sigmoid'),
    tf.keras.layers.Dense(10, activation='softmax')
])


In [ ]:
# Célula 4: Compile o modelo
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.1), loss=tf.keras.losses.MeanSquaredError(), metrics=['accuracy'])

# Treine o modelo
model.fit(data.train_dataloader(), epochs=10)


In [ ]:
# Célula 5: Avalie a acurácia no conjunto de validação
test_acc = d2l.evaluate_accuracy(model, data.val_dataloader())
print(f"Acurácia no conjunto de validação: {test_acc}")


In [ ]:
# Célula 6: Visualização dos exemplos mal-classificados
misclassified_indices = []

for i, (X, y) in enumerate(data.val_dataloader()):
    predictions = model(X)
    predicted_labels = tf.argmax(predictions, axis=1)
    incorrect_indices = tf.where(predicted_labels != y).numpy()
    incorrect_indices += i * data.batch_size
    misclassified_indices.extend(incorrect_indices)

# Exibir alguns exemplos mal-classificados
num_examples_to_display = 5
for i in range(num_examples_to_display):
    index = misclassified_indices[i]
    misclassified_image = data.val[0][index]
    true_label = data.val[1][index]
    predicted_label = predicted_labels[index].numpy()
    
    plt.imshow(misclassified_image, cmap='gray')
    plt.title(f'True Label: {true_label}, Predicted Label: {predicted_label}')
    plt.show()


In [ ]:
# Célula 7: Matriz de Confusão
from sklearn.metrics import confusion_matrix

true_labels = []
predicted_labels = []

for X, y in data.val_dataloader():
    predictions = model(X)
    predicted_labels.extend(tf.argmax(predictions, axis=1).numpy())
    true_labels.extend(y.numpy())

confusion = confusion_matrix(true_labels, predicted_labels)

# Exibir a matriz de confusão
print("Matriz de Confusão:")
print(confusion)